### Google calendar como tool

Instalar la librería de Google Client

In [ ]:
#se instala usando poetry
#poetry install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
#import básicas
import datetime
import os
import io  # Para manejo de JSON

In [39]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Scope para acceso de lectura/escritura en Calendar
SCOPES = ['https://www.googleapis.com/auth/calendar']

def main():
    creds = None
    token_google = 'token_google'  # Ajusta si es diferente (ej., 'token.json')
    credentials_file = '../credentials.json'  # Tu archivo de credenciales
    
    # Verifica si el archivo de token existe y es válido
    if os.path.exists(token_google):
        try:
            creds = Credentials.from_authorized_user_file(token_google, SCOPES)
            print("Token cargado correctamente.")
        except ValueError as e:
            print(f"Error en el token: {e}. Borrando y regenerando...")
            os.remove(token_google)  # Borra automáticamente el archivo malo
            creds = None  # Fuerza reautorización
    
    # Si no hay credenciales válidas, autoriza
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            if not os.path.exists(credentials_file):
                print(f"Error: No se encuentra {credentials_file}. Descárgalo de Google Cloud Console.")
                return
            flow = InstalledAppFlow.from_client_secrets_file(credentials_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # Guarda las credenciales para la próxima vez
        with open(token_google, 'w') as token:
            token.write(creds.to_json())
        print(f"Token guardado en {token_google}.")
    
    try:
        service = build('calendar', 'v3', credentials=creds)
        
        # Configura el evento básico (ajusta la fecha si quieres)
        event = {
            'summary': 'Reunión de Prueba con IA',
            'location': 'Oficina Virtual (Zoom)',
            'description': 'Evento creado con Python y Google Calendar API.',
            'start': {
                'dateTime': '2025-09-15T10:00:00-05:00',  # Mañana a las 10 AM (ajusta)
                'timeZone': 'America/Guayaquil',  # Timezone Ecuador
            },
            'end': {
                'dateTime': '2025-09-15T11:00:00-05:00',
                'timeZone': 'America/Guayaquil',
            },
            'attendees': [
                {'email': 'tu-email@ejemplo.com'},  # Agrega emails reales
            ],
            'reminders': {
                'useDefault': False,
                'overrides': [
                    {'method': 'email', 'minutes': 24 * 60},  # Email 1 día antes
                    {'method': 'popup', 'minutes': 10},       # Popup 10 min antes
                ],
            },
        }
        
        # Crea el evento en el calendario principal
        event = service.events().insert(calendarId='primary', body=event).execute()
        print(f'Evento creado exitosamente: {event.get("htmlLink")}')
        
    except HttpError as error:
        print(f'Error al crear el evento: {error}')

if __name__ == '__main__':
    main()

Token cargado correctamente.
Token guardado en token_google.
Evento creado exitosamente: https://www.google.com/calendar/event?eid=bjAwc2dya2xiNzVlNTM5amdxYmJkM3M0NHMgamVmZXJzb25ndWFtYW45NEBt


In [38]:
service = build('calendar', 'v3', credentials=creds)

event = {
  'summary': 'Renunion de agentes de IA',
  'location': 'Oficina principal',
  'description': 'Asistirir formal',
  'start': {
    'dateTime': '2015-09-14T09:00:00-07:00',
    'timeZone': 'America/Guayaquil',
  },
  'end': {
    'dateTime': '2015-09-14T09:20:00-07:00',
    'timeZone': 'America/Guayaquil',
  },
  'attendees': [
    {'email': 'lpage@example.com'},
    {'email': 'sbrin@example.com'},
  ],
  'reminders': {
    'useDefault': False,
    'overrides': [
      {'method': 'email', 'minutes': 24 * 60},
      {'method': 'popup', 'minutes': 10},
    ],
  },
}

event = service.events().insert(calendarId='primary', body=event).execute()
print('Event created: %s' % (event.get('htmlLink')))


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.